In [1]:
%pip install kneed

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/sw/arch/RHEL8/EB_production/2022/software/Python/3.10.4-GCCcore-11.3.0/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
csv_file_path = 'elbow_avg_output_sample0.csv'
data = pd.read_csv(csv_file_path, header=0)
print(data)
csv_file_path = 'init_input_all.csv'
sentences = pd.read_csv(csv_file_path, header=0)
print(sentences)

         0  1        2         3         4
0        0  2   German  0.006339  0.007812
1        1  2   German  0.006247  0.007812
2        2  2   German  0.006243  0.007812
3        3  2   German  0.006420  0.007812
4        4  2   German  0.005457  0.007812
...    ... ..      ...       ...       ...
46447  469  0  Chinese  0.004766  0.054688
46448  470  0  Chinese  0.005004  0.054688
46449  471  0  Chinese  0.005675  0.054688
46450  472  0  Chinese  0.006226  0.054688
46451  473  0  Chinese  0.006332  0.054688

[46452 rows x 5 columns]
      ID  Type                                             Source
0      0     2  Andrei picked up the chair or the bag and the ...
1      1     2  Danny held the chair or the bag and the telescope
2      2     2  Danny moved the chair or the bag and the teles...
3      3     2  Andrei held the telescope or the chair and the...
4      4     2    Andrei and Danny moved the yellow bag and chair
..   ...   ...                                                

In [3]:
import kneed
import numpy as np
import matplotlib.pyplot as plt

results = np.empty((0, 6))
for id in data['0'].unique():
    for lang in data['2'].unique():
      filter = data[(data['0']==id) & (data['2']==lang)]
      t = filter['1'].unique()[0]
      e = filter['3']
      c = filter['4']
      kneedle = kneed.KneeLocator(c, e, S=1.0, curve="convex", direction="decreasing")
      x = kneedle.knee
      y = kneedle.knee_y
      try:
        r = 1 / (kneedle.knee_y / kneedle.knee) ** 0.5
      except Exception as e:
        r = 0
      new_row = np.array([id, t, lang, x, y, r])
      results = np.append(results, [new_row], axis=0)

      if id == 0:
        if False: #lang == 'Persian':
          kneedle.plot_knee_normalized(title='Knee chart for sentence %s (%s)\nknee=(%s,%s)' % (id, lang, x, y))
          plt.savefig('knee_%s_%s.png' % (id, lang))
print(results)

[['0' '2' 'German' '0.0390625' '0.004826358' '2.8449212605534058']
 ['0' '2' 'Greek' '0.03125' '0.0050666444' '2.483503626675396']
 ['0' '2' 'Spanish' '0.0234375' '0.005459119' '2.0720219980329957']
 ...
 ['473' '0' 'Russian' '0.03125' '0.0054762126' '2.38882788933731']
 ['473' '0' 'Turkish' '0.015625' '0.0066272183' '1.535480745152257']
 ['473' '0' 'Chinese' '0.015625' '0.0067219124' '1.5246269377063748']]


In [4]:
import pandas as pd
import matplotlib.colors as mcolors

plt.clf()

df = pd.DataFrame(results)
df[0] = df[0].astype(int)
df[1] = df[1].astype(int)
df[2] = df[2].astype(str)
df[3] = df[3].astype(float)
df[4] = df[4].astype(float)
df[5] = df[5].astype(float)

# Get unique categories
unique_categories = np.unique(df[2].unique())
num_categories = len(unique_categories)

# Define a colormap with enough distinct colors
cmap = plt.get_cmap('tab10', num_categories)
norm = mcolors.Normalize(vmin=0, vmax=num_categories - 1)
sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])

scatter_plots = {}
for i, lang in enumerate(df[2].unique()):
    df2 = df[(df[2] == lang)]
    data = {'X': df2[3].values,
            'Y': df2[4].values}
    df2 = pd.DataFrame(data)
    scatter_plots[lang] = plt.scatter(df2['X'], df2['Y'], marker='o', color=cmap(i), label=lang)
for i, label in enumerate(df[0]):
    plt.annotate(label, (df[3][i], df[4][i]), textcoords="offset points",
                 xytext=(5,5), ha='center')
plt.xlabel('Knee Complexity')
plt.ylabel('Knee error')
plt.title('Knee points')
plt.legend(handles=[scatter_plots[cat] for cat in unique_categories])
plt.grid(True)
plt.savefig('knee_scatter.png')
plt.show()

In [5]:
df[6] = np.sqrt((1-df[3]) * (1-df[4]))
print(df)

        0  1         2         3         4         5         6
0       0  2    German  0.039062  0.004826  2.844921  0.977906
1       0  2     Greek  0.031250  0.005067  2.483504  0.981754
2       0  2   Spanish  0.023438  0.005459  2.072022  0.985511
3       0  2   Persian  0.007812  0.006292  1.114314  0.992948
4       0  2    French  0.007812  0.006799  1.071982  0.992694
...   ... ..       ...       ...       ...       ...       ...
6631  473  0     Dutch  0.031250  0.006047  2.273273  0.981271
6632  473  0  Romanian  0.031250  0.006052  2.272369  0.981268
6633  473  0   Russian  0.031250  0.005476  2.388828  0.981552
6634  473  0   Turkish  0.015625  0.006627  1.535481  0.988864
6635  473  0   Chinese  0.015625  0.006722  1.524627  0.988817

[6636 rows x 7 columns]


In [7]:
alphas = np.empty((0,5))
for id in df[0].unique():
  filter = df[df[0]==id]
  filter[7]=2*np.abs(filter[5]-0.5)
  filter = filter.reset_index(drop=True)
  n=len(filter[2].unique())
  mx = np.empty((n,n))
  for i in range(n):
    for j in range(n):
      mx[i,j]=np.abs(filter[5][i]-filter[5][j])
  term1 = 1 - np.sum(filter[5]) / n
  term2 = np.sum(mx) / (n**2)
  alpha = 0.5 * term1 + 0.5 * term2
  alphas = np.vstack((alphas, [id, filter[1][0], term1, term2, alpha]))
df_alphas = pd.DataFrame(alphas)
df_alphas[0] = df_alphas[0].astype(int)
df_alphas[1] = df_alphas[1].astype(int)
df_alphas[2] = df_alphas[2].astype(float)
df_alphas[3] = df_alphas[3].astype(float)
df_alphas[4] = df_alphas[4].astype(float)

/scratch-local/bmehrparvar.5983040/ipykernel_1544898/4164460169.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filter[7]=2*np.abs(filter[5]-0.5)


In [8]:
merged_df = pd.merge(df_alphas, sentences, left_on=0, right_on='ID')
merged_df = merged_df.loc[:,[0,1,2,3,4,'Source']]
merged_df.columns = ['ID', 'type', 'term1', 'term2', 'alpha', 'sentence']
print(merged_df)
merged_df.to_csv('alphas_avg_output_sample0_curve.csv', index=False)

      ID  type     term1     term2     alpha  \
0      0     2 -0.715484  0.862089  0.073302   
1      1     2 -0.885164  0.704113 -0.090526   
2      2     2 -0.957343  0.825653 -0.065845   
3      3     2 -0.846041  0.689026 -0.078507   
4      4     2 -1.213531  0.837247 -0.188142   
..   ...   ...       ...       ...       ...   
469  469     0 -0.876034  1.127153  0.125559   
470  470     0 -0.617195  0.529961 -0.043617   
471  471     0 -1.098233  0.571645 -0.263294   
472  472     0 -0.509381  0.735459  0.113039   
473  473     0 -1.080198  0.570384 -0.254907   

                                              sentence  
0    Andrei picked up the chair or the bag and the ...  
1    Danny held the chair or the bag and the telescope  
2    Danny moved the chair or the bag and the teles...  
3    Andrei held the telescope or the chair and the...  
4      Andrei and Danny moved the yellow bag and chair  
..                                                 ...  
469       Andrei and Dan